### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_multiple_reports
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20221004_scenario_projections.ipynb")

2022/09/27 12:30:27 INFO|============================================================


### Fit an initial scenario through February 2022

In [3]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20221004_oct_gov_briefing/20221004_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/input/vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2022-12-31',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/09/27 12:30:54 INFO|-------------------------|{} Retrieving vaccinations data
2022/09/27 12:30:56 INFO|-------------------------|{} Constructing vaccination projections
2022/09/27 12:30:57 INFO|-------------------------|{} Retrieving hospitalizations data
2022/09/27 12:30:58 INFO|----------------------|{} Prepping Model
2022/09/27 12:30:58 INFO|-----------------------|{} Prepping Model
2022/09/27 12:32:00 INFO|----------------------|{} Model prepped for fitting in 61.681910417 seconds.
2022/09/27 12:32:00 INFO|----------------------|{} Will fit 25 times
2022/09/27 12:33:11 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/25 completed in 70.39958854100001 seconds: {0: {'co': 0.8587936430054103}, 14: {'co': 0.6897532925696396}, 28: {'co': 0.552085709918695}, 42: {'co': 0.001266490224538246}, 56: {'co': 0.6815608359848625}, 70: {'co': 0.8912262029490517}}
2022/09/27 12:33:11 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/09/27 12:3

2022/09/27 12:38:33 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/25 completed in 12.549672208000004 seconds: {504: {'co': 0.7861747076322041}, 518: {'co': 0.7799119008001745}, 532: {'co': 0.7570420612949874}, 546: {'co': 0.7091680479700163}, 560: {'co': 0.7430672460785883}, 574: {'co': 0.7376073419647999}}
2022/09/27 12:38:33 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/09/27 12:38:50 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/25 completed in 15.120841207999945 seconds: {532: {'co': 0.7562168467677969}, 546: {'co': 0.7112534574467083}, 560: {'co': 0.7393398150748843}, 574: {'co': 0.749263265202643}, 588: {'co': 0.7862516213289434}, 602: {'co': 0.7652609952603614}}
2022/09/27 12:38:50 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/09/27 12:39:03 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/25 completed in 11.851243833000012 seconds: 

### Create and run scenarios from Feb 2022 to present

In [5]:
multiprocess = 4

scenario_params = json.load(open("covid_model/analysis/20221004_oct_gov_briefing/20221004_scenario_params.json"))

model_args = {
    'base_spec_id': 4021, #model.spec_id, # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# define vaccine effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for vx_seed in [0, 1]:
    for vir_mult in [0.833, 2.38]:
        hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*0.8),
                   "2022-03-15": (0.34 + 0.66*0.8), "2022-03-30": 0.8}
        vx_adjust = [{"param": "vx_seed",
                      "vals": {"2020-01-01": 0, "2022-09-30": vx_seed, "2022-10-20": 0},
                      "desc": "Variant X seeding"}]
        vir_adjust = [{"param": "hosp",
                       "attrs": {"variant": "vx"},
                       "mults": {"2020-01-01": vir_mult},
                       "desc": "Variant X hospitalization multiplier"}]
        lt5_vacc_adjust = [{"param": "immunity",
                                "attrs": {'age': '0-19', 'vacc': 'shot1'},
                                "mults": {"2020-01-01": 1, "2022-06-20": 0.99 + 0.01*vacc_eff_lt5,
                                          "2022-06-30": 0.98 + 0.02*vacc_eff_lt5,
                                          "2022-07-10": 0.97 + 0.03*vacc_eff_lt5,
                                          "2022-07-20": 0.96 + 0.04*vacc_eff_lt5,
                                          "2022-07-30": 0.955 + 0.045*vacc_eff_lt5,
                                          "2022-08-10": 0.95 + 0.05*vacc_eff_lt5,
                                          "2022-08-20": 0.945 + 0.055*vacc_eff_lt5,
                                          "2022-08-30": 0.94 + 0.06*vacc_eff_lt5,
                                          "2022-09-10": 0.935 + 0.065*vacc_eff_lt5,
                                          "2022-09-20": 0.93 + 0.07*vacc_eff_lt5},
                                "desc": "weighted average using share of 0-19 getting shot1 who are under 5"}]
        scenario_model_args.append({'params_defs': scenario_params + vx_adjust + vir_adjust + lt5_vacc_adjust,
                                        'hosp_reporting_frac': hrf,
                                        'tags': {'vx_seed': vx_seed,
                                                 'vir_mult': vir_mult}})
            

In [6]:
# check how many scenarios there are
len(scenario_model_args)

4

In [7]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/09/27 12:46:40 INFO|============================================================
2022/09/27 12:46:40 INFO|============================================================
2022/09/27 12:46:40 INFO|============================================================
2022/09/27 12:46:40 INFO|============================================================
2022/09/27 12:46:43 INFO|-----------|{'vx_seed': 1, 'vir_mult': 0.833} Retrieving vaccinations data
2022/09/27 12:46:43 INFO|-----------|{'vx_seed': 1, 'vir_mult': 2.38} Retrieving vaccinations data
2022/09/27 12:46:43 INFO|-----------|{'vx_seed': 0, 'vir_mult': 0.833} Retrieving vaccinations data
2022/09/27 12:46:43 INFO|-----------|{'vx_seed': 0, 'vir_mult': 2.38} Retrieving vaccinations data
2022/09/27 12:46:45 INFO|-----------|{'vx_seed': 0, 'vir_mult': 2.38} Constructing vaccination projections
2022/09/27 12:46:45 INFO|-----------|{'vx_seed': 1, 'vir_mult': 0.833} Constructing vaccination projections
2022/09/27 12:46:45 INFO|-----------|{'vx_s

2022/09/27 12:49:00 INFO|--------|{'vx_seed': 1, 'vir_mult': 0.833, 'fit_batch': '3'}: Transmission control fit 4/5 completed in 13.173636208000005 seconds: {851: {'co': 0.8320717317577984}, 865: {'co': 0.8605611687442443}, 879: {'co': 0.8536895362340359}, 893: {'co': 0.8540746048955223}, 907: {'co': 0.8616319705300491}, 921: {'co': 0.8574293174277589}}
2022/09/27 12:49:00 INFO|---------|{'vx_seed': 1, 'vir_mult': 0.833, 'fit_batch': '3'}: Running forward sim
2022/09/27 12:49:00 INFO|--------|{'vx_seed': 1, 'vir_mult': 2.38, 'fit_batch': '3'}: Transmission control fit 4/5 completed in 13.068495374999998 seconds: {851: {'co': 0.8320717317577984}, 865: {'co': 0.8605611687442443}, 879: {'co': 0.8536895362340359}, 893: {'co': 0.8540746048955223}, 907: {'co': 0.8616319705300491}, 921: {'co': 0.8574293174277589}}
2022/09/27 12:49:00 INFO|---------|{'vx_seed': 1, 'vir_mult': 2.38, 'fit_batch': '3'}: Running forward sim
2022/09/27 12:49:01 INFO|--------|{'vx_seed': 0, 'vir_mult': 2.38, 'fit_ba

2022/09/27 12:49:18 INFO|--------|{'vx_seed': 0, 'vir_mult': 2.38, 'fit_batch': '4', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.858793642028981}, 14: {'co': 0.6897532959819603}, 28: {'co': 0.5462101837436237}, 42: {'co': 1.4641814421628474e-14}, 56: {'co': 0.6964651010113871}, 70: {'co': 0.8333063012789972}, 84: {'co': 0.8295949715042341}, 98: {'co': 0.8317180559435786}, 112: {'co': 0.8774454677720657}, 126: {'co': 0.8243715551500813}, 140: {'co': 0.7604768988940638}, 154: {'co': 0.6712541469637572}, 168: {'co': 0.8430274766789462}, 182: {'co': 0.7826503998325627}, 196: {'co': 0.8191652898178112}, 210: {'co': 0.7726023558330771}, 224: {'co': 0.7713430227191537}, 238: {'co': 0.7185520635395145}, 252: {'co': 0.7221395050842289}, 266: {'co': 0.6850199040795334}, 280: {'co': 0.7198410203796914}, 294: {'co': 0.7578493583347318}, 308: {'co': 0.822592721511341}, 322: {'co': 0.8115175700835301}, 336: {'co': 0.7825628118439047}, 350: {'co': 0.7953867642261275}, 364: {'co': 0.8054764773329308},

### Run the report for each fit model

In [8]:
# here you can also specify which variants you want to calculate immunity for
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir,
                           prep_model=False, solve_model=True, immun_variants=['ba45', 'vx'], from_date='2022-01-01')

2022/09/27 12:50:53 INFO|============================================================
2022/09/27 12:50:53 INFO|--------|Solving model
2022/09/27 12:50:53 INFO|============================================================
2022/09/27 12:50:53 INFO|--------|Solving model
2022/09/27 12:50:54 INFO|============================================================
2022/09/27 12:50:54 INFO|--------|Solving model
2022/09/27 12:50:54 INFO|============================================================
2022/09/27 12:50:54 INFO|--------|Solving model
2022/09/27 12:50:56 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/09/27 12:50:56 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/09/27 12:50:57 WARNING|No artists with labels found to put in legend.  Note that artists 

In [9]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-12-31', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-12-31', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/09/27 12:54:07 INFO|Projecting
2022/09/27 12:54:07 INFO|
2022/09/27 12:54:21 INFO|{'vx_seed': 0, 'vir_mult': 0.833, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/27 12:54:23 INFO|
2022/09/27 12:54:36 INFO|{'vx_seed': 0, 'vir_mult': 2.38, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/27 12:54:38 INFO|
2022/09/27 12:54:52 INFO|{'vx_seed': 1, 'vir_mult': 0.833, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/27 12:54:53 INFO|
2022/09/27 12:55:07 INFO|{'vx_seed': 1, 'vir_mult': 2.38, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/27 12:55:08 INFO|Running reports
